# Anthropic Claude

In this notebook, we demonstrate how a to use Anthropic Claude model for AgentChat.

## Requirements
To use Anthropic Claude with AutoGen, first you need to install the `pyautogen` and `anthropic` package.


In [ ]:
!pip install pyautogen anthropic

In [2]:
import inspect
from typing import Any, Dict, List, Union

from anthropic import Anthropic
from anthropic.types import Completion, Message

import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.oai.openai_utils import OAI_PRICE1K

## Create Anthropic Model Client following ModelClient Protocol

We will implement our Anthropic client adhere to the `ModelClient` protocol and response structure which is defined in client.py and shown below.


```python
class ModelClient(Protocol):
    """
    A client class must implement the following methods:
    - create must return a response object that implements the ModelClientResponseProtocol
    - cost must return the cost of the response
    - get_usage must return a dict with the following keys:
        - prompt_tokens
        - completion_tokens
        - total_tokens
        - cost
        - model

    This class is used to create a client that can be used by OpenAIWrapper.
    The response returned from create must adhere to the ModelClientResponseProtocol but can be extended however needed.
    The message_retrieval method must be implemented to return a list of str or a list of messages from the response.
    """

    RESPONSE_USAGE_KEYS = ["prompt_tokens", "completion_tokens", "total_tokens", "cost", "model"]

    class ModelClientResponseProtocol(Protocol):
        class Choice(Protocol):
            class Message(Protocol):
                content: Optional[str]

            message: Message

        choices: List[Choice]
        model: str

    def create(self, params) -> ModelClientResponseProtocol:
        ...

    def message_retrieval(
        self, response: ModelClientResponseProtocol
    ) -> Union[List[str], List[ModelClient.ModelClientResponseProtocol.Choice.Message]]:
        """
        Retrieve and return a list of strings or a list of Choice.Message from the response.

        NOTE: if a list of Choice.Message is returned, it currently needs to contain the fields of OpenAI's ChatCompletion Message object,
        since that is expected for function or tool calling in the rest of the codebase at the moment, unless a custom agent is being used.
        """
        ...

    def cost(self, response: ModelClientResponseProtocol) -> float:
        ...

    @staticmethod
    def get_usage(response: ModelClientResponseProtocol) -> Dict:
        """Return usage summary of the response using RESPONSE_USAGE_KEYS."""
        ...
```


## Implementation of AnthropicClient

You can find the introduction to Claude-3-Opus model [here](https://docs.anthropic.com/claude/docs/intro-to-claude). 

Since anthropic provides their Python SDK with similar structure as OpenAI's, we will following the implementation from `autogen.oai.client.OpenAIClient`.



In [3]:
TOOL_ENABLED = False


class AnthropicClient:
    def __init__(self, config: Dict[str, Any]):
        self._config = config
        self.model = config["model"]
        anthropic_kwargs = set(inspect.getfullargspec(Anthropic.__init__).kwonlyargs)
        filter_dict = {k: v for k, v in config.items() if k in anthropic_kwargs}
        self._client = Anthropic(**filter_dict)

    def message_retrieval(self, response: Message) -> Union[List[str], List]:
        """Retrieve the messages from the response."""
        choices = response.content
        if isinstance(response, Message):
            return [choice.text for choice in choices]  # type: ignore [union-attr]

        # claude python SDK and API not yet support function calls

    def create(self, params: Dict[str, Any]) -> Completion:
        """Create a completion for a given config using openai's client.

        Args:
            client: The openai client.
            params: The params for the completion.

        Returns:
            The completion.
        """
        if "messages" in params:
            raw_contents = params["messages"]
            if raw_contents[0]["role"] == "system":
                raw_contents = raw_contents[1:]
                params["messages"] = raw_contents
            completions: Completion = self._client.messages  # type: ignore [attr-defined]
        else:
            completions: Completion = self._client.completions

        # Not yet support stream
        params = params.copy()
        params["stream"] = False
        params.pop("model_client_cls")
        response = completions.create(**params)

        return response

    def cost(self, response: Completion) -> float:
        """Calculate the cost of the response."""
        total = 0.0
        tokens = {
            "input": response.usage.input_tokens if response.usage is not None else 0,
            "output": response.usage.output_tokens if response.usage is not None else 0,
        }
        price_per_million = {
            "input": 15,
            "output": 75,
        }
        for key, value in tokens.items():
            total += value * price_per_million[key] / 1_000_000

        return total

    @staticmethod
    def get_usage(response: Completion) -> Dict:

        return {
            "prompt_tokens": response.usage.input_tokens if response.usage is not None else 0,
            "completion_tokens": response.usage.output_tokens if response.usage is not None else 0,
            "total_tokens": (
                response.usage.input_tokens + response.usage.output_tokens if response.usage is not None else 0
            ),
            "cost": response.cost if hasattr(response, "cost") else 0,
            "model": response.model,
        }

## Set the config for the Anthropic API

You can add any parameters that are needed for the custom model loading in the same configuration list.

It is important to add the `model_client_cls` field and set it to a string that corresponds to the class name: `"CustomModelClient"`.

In [10]:
import os

config_list_claude = [
    {
        # Choose your model name.
        "model": "claude-3-opus-20240229",
        # You need to provide your API key here.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "base_url": "https://api.anthropic.com",
        "api_type": "anthropic",
        "model_client_cls": "AnthropicClient",
    }
]

## Construct Agents

Construct a simple conversation between a User proxy and an ConversableAgent based on Claude-3 model.


`max_tokens` argument is mandatory in the `llm_config`.

In [11]:
assistant = AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_claude,
        "max_tokens": 100,
    },
)
user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config=False,
)

[autogen.oai.client: 04-02 22:48:52] {418} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.


## Register the custom client class to the assistant agent

In [12]:
assistant.register_model_client(model_client_cls=AnthropicClient)

In [13]:
user_proxy.initiate_chat(
    assistant,
    message="Who are you?",
)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to assistant):

Who are you?

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


assistant (to user_proxy):

I am an artificial intelligence called Claude. I was created by Anthropic to be an intelligent conversational assistant, but I'm not a real person.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


ChatResult(chat_id=None, chat_history=[{'content': 'Who are you?', 'role': 'assistant'}, {'content': "I am an artificial intelligence called Claude. I was created by Anthropic to be an intelligent conversational assistant, but I'm not a real person.", 'role': 'user'}], summary="I am an artificial intelligence called Claude. I was created by Anthropic to be an intelligent conversational assistant, but I'm not a real person.", cost=({'total_cost': 0, 'claude-3-opus-20240229': {'cost': 0, 'prompt_tokens': 11, 'completion_tokens': 34, 'total_tokens': 45}}, {'total_cost': 0, 'claude-3-opus-20240229': {'cost': 0, 'prompt_tokens': 11, 'completion_tokens': 34, 'total_tokens': 45}}), human_input=['exit'])